In [ ]:
"""
Get articles/texts, and filter only sentences with white-listed vocab OR named entities.

Contents:
-load packages
-create helper functions
-data intialization from csv file of words &
 extend vocab set
-build data structure to house info moving forward
-write to .json formats
-read from existing .json formats
-get white-listed of words and inflections
-scrape journale en francais facile
"""

In [83]:
import bs4, requests, sys, codecs, urllib.request, re
from bs4 import SoupStrainer
from bs4.element import Comment
import random
import string
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import mlconjug3
import sklearn
import goslate
from nltk.tokenize import word_tokenize, sent_tokenize
import pprint
pp = pprint.PrettyPrinter()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)
import mitosheet
import translators as ts

In [3]:
def add_noun(add_word):

    if add_word[len(add_word)-1] != 's' and add_word[len(add_word)-1] != 'x':

        out_word = add_word + 's'

        return out_word
    else:
        return add_word

In [84]:
##METHODS

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = bs4.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers={'User-Agent':user_agent,}


In [5]:
def get_known(unknown_list):
    
    count_got = 0
    known_list = []
    count_checked = 0
    for word in unknown_list:
        count_checked += 1
        decision = str(input(word+"\nKnown =k"))
        if decision =='k':
            known_list.append(word)
            count_got +=1
            print("got "+str(count_got))
        elif decision=='q':
            break
        print(str(len(unknown_list)-count_checked)+' remaining')
    return known_list

In [6]:
def review_out(out_dict,prev_learned,eb_unk_in,eb_unk_m_in,examples):
    """
    out_dict = out
    prev_learned = eb_learned
    eb_unk_in =  eb_unk
    eb_unk_m_in = eb_unk_morph
    examples = eb_unk_examples
    
    decisions:
    k = word known, remove from eb_unk and eb_unk_morph
        and add to eb_learned
        
    m = remove morph from eb_unk and eb_unk_morph 
    
    a = add example to eb_unk_examples
    q = quit for day
    """
    for word in out_dict.keys():
        print("Lemma: "+word)
        print("Morph: "+str(out_dict[word]['morph']))
        
        if len(examples[word])>0:
            print("Previous examples: ")
            for sent in examples[word]:
                print(" - "+sent)
                    
        print("New examples: ")
        
        for sent in out_dict[word]['sents']:
            if sent not in examples[word]:
                print(sent)
                decision = str(input("options : k,m,a,q"))
                if decision =='k':
                    try:
                        examples[word].append(sent) #by default adds examples
                        prev_learned[word] = eb_unk_in[word].copy()
                        del eb_unk_in[word] #remove lemma

                        i = 0
                        while i < len(eb_unk_m_in):

                            if eb_unk_m_in[list(eb_unk_m_in)[i]] == word:
                               del eb_unk_m_in[list(eb_unk_m_in)[i]]
                            else:
                                i +=1
                    except:
                        pass

                if decision =='m':
                    morph = str(input("which morph:"))
                    try:
                        del eb_unk_m_in[morph]
                        eb_unk_in[word].remove(morph) 
                    except:
                        pass
                if decision =='a':
                    try:
                        examples[word].append(sent)
                    except:
                        pass
                if decision =='q':
                    break
    

In [7]:
def how_to_add(word):
    """
    p = plain, add without conjurgating or changing
    c = conjugate as normal verb
    x = noun that adds x when plural
    s = noun that add s when plural
    iv = inner verb. There is a verb in the expression that should be 
        conjurgated
    """
    extras_morphs = []
    extras_morphs.append(word)
    
    
    gs = goslate.Goslate()

    default_conjugator = mlconjug3.Conjugator(language='fr')

    decision = str(input(word+"\nc\nx\ns\niv"))
    
    if decision == 'c':

        try:
            temp = []
            test_verb = default_conjugator.conjugate(word)
            all_conjugated_forms = test_verb.iterate()
            for item in all_conjugated_forms:
                if item not in temp:
                    temp.append(item[len(item)-1])
            extras_morphs.extend(list(set(temp)))
        except:
            pass
    elif decision == 'x':
        extras_morphs.append(lemma+'x')
    elif decision == 's':
        extras_morphs.append(lemma+'s')
    elif decision == 'iv':
        morph = str(input("which morph:"))
        

        temp = []
        test_verb = default_conjugator.conjugate(morph)
        all_conjugated_forms = test_verb.iterate()
        for item in all_conjugated_forms:
            if item not in temp:
                temp.append(item[len(item)-1])

        for variant in list(set(temp)):
            if variant is not None:
                new_word = word.replace(morph,variant)
                extras_morphs.append(new_word)

    return extras_morphs

In [85]:
"""
For eb_unk words: search through all articles, return 
sentences containing any words for unknown vocabulary. 

return dict object, key is lemma, then values are morphs found
and array of examples. 
"""

def find_unks(webpages,eb_unk_mo,prev_examples):
    
    output = {}
    for webpage in webpages:
        try:

            request=urllib.request.Request(webpage,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read()
            contents = text_from_html(data)

            contents_array = sent_tokenize(contents)

            for line in contents_array:
                tokenized = word_tokenize(line, language='french')
                for word in tokenized:
                    if word.lower() in eb_unk_mo:

                        if word.lower() in prev_examples:
                            if line not in prev_examples[word.lower()]:
                                if word.lower() not in output:
                                    output[eb_unk_mo[word.lower()]] = {} #key will be lemma
                                    output[eb_unk_mo[word.lower()]]['morph'] = []
                                    output[eb_unk_mo[word.lower()]]['sents'] = []

                                output[eb_unk_mo[word.lower()]]['morph'].append(word.lower())
                                output[eb_unk_mo[word.lower()]]['sents'].append(line)

        except:
            pass
    return output
    

In [9]:
def filter_text(webpage,text=None,print_word_lvl=False):


    #try:
    if text is None:
        request=urllib.request.Request(webpage,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
    else:
        contents = webpage

    line_array = []
    percent_array = []
    contents_array = sent_tokenize(contents)

    disallowed_words = set()
    total_words = 0
    unknown_words = 0
    lines = []
    unknowns = []
    
    for line in contents_array:
        
        line_total = 0
        line_unks = 0
        tokenized = word_tokenize(line, language='french')

        unk_str = ""
        for word in tokenized:
            total_words +=1
            line_total +=1

            if not (bool(re.search("[0-9]", word)) or\
                    bool(re.search("[A-Z]", word)) or\
                    bool(re.search("[.,\/#!$%\^&\*;:{}=\-_`~()«»]", word)) or\
                    word.lower() in vocab['white_listed']):
                if not ((word.lower() in vocab['white_listed']) or (word.lower() in vocab['black_listed'])):
                    unk_str += '"'+word.lower()+'"'+", "

                disallowed_words.add(word.lower())
                unknown_words +=1
                line_unks +=1


        line_array.append(line)
        line_percent = (line_total-line_unks)/line_total
        percent_array.append(line_percent)

        if len(unk_str)>0:
            unk_str = unk_str[0:len(unk_str)-1]

        unknowns.append(unk_str)

    if print_word_lvl and total_words>0:
        print("word-level % known = "+str((1-(unknown_words/total_words))*100))

    return_pd = pd.DataFrame(list(zip(line_array,percent_array,unknowns)))
    return_pd.columns = ["line","line_percent","unk_words"]

    with open(path+'unknown_french_dad_list.txt',"w") as outfile:
        outfile.write(str(list(disallowed_words)))

    return return_pd
    
    #except:
    #    pass

##Dad section

In [13]:
path = "/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/"
source_file = "french_dad.json"

In [14]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13376


In [11]:
            
##Journal en francais facile:
print("Checking Journal en francais facile")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://francaisfacile.rfi.fr/fr/podcasts/journal-en-fran%c3%a7ais-facile/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))


pages = []
for link in soup.find_all('a', href=True):
    if '/fr/podcasts/le-journal-en-fran%C3%A7ais-facile/' in str(link['href']):
        #print(str(link['href']))
        str_page = 'https://savoirs.rfi.fr/'+str(link['href'])
        #str_page = 'https://savoirs.rfi.fr/en/apprendre-enseigner/langue-francaise/journal-en-francais-facile-'+str(re.search(r"[0-9]{8}.*",str(link['href'])).group(0))
        if str_page not in pages:
            pages.append(str_page)

pages

Checking Journal en francais facile


['https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221123-kiev-priv%C3%A9e-d-%C3%A9lectricit%C3%A9-cyberattaque-au-parlement-europ%C3%A9en-l-affaire-adrien-quatennens',
 'https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221122-france-les-magistrats-sont-en-gr%C3%A8ve-kherson-bombard%C3%A9e-par-les-russes-le-mondial-de-foot',
 'https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221121-mondial-de-foot-tortures-en-ukraine-chol%C3%A9ra-au-liban-ch%C3%B4mage-en-france',
 'https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221118-la-cor%C3%A9e-du-nord-tire-un-missile-la-cop27-black-ad-remporte-le-prix-d%C3%A9couvertes-rfi',
 'https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221117-600-prisonniers-lib%C3%A9r%C3%A9s-en-birmanie-l-ukraine-bombard%C3%A9e-que-fait-emmanuel-macron-%C3%A0-bangkok',
 'https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221116-la-fu

In [15]:
with open(path+"text_input_web_dad_fr.txt","r") as infile:
    textfile = infile.read()
filtered_art = filter_text(textfile,text= True)
#display(filtered_art)

In [16]:
all_unks = set()

with open(path+'unknown_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [re.sub("^['|\"]|^\['|['|\"]$|'\]$|\[","",line.strip().lower()) for line in new_words] #update regex
    for word in new_words:
        all_unks.add(word)
len(all_unks)

154

In [17]:
all_unks=list(all_unks)
unk_df = pd.DataFrame(all_unks)
unk_df.columns = ['word']
unk_df['status'] = pd.Series(['' for word in all_unks])

In [18]:
mitosheet.sheet(unk_df, analysis_to_replay="id-xgjqdbiwry")

MitoWidget(analysis_data_json='{"analysisName": "id-xgjqdbiwry", "analysisToReplay": null, "code": [], "stepSu…

In [19]:
add_ints = [0,8,15,18,20,23,26,40,42,43,50,51,52,53,56,58,62,68,70,72,76,78,80,81,82,83,84,85,89,92,86,87,99,101,104,107,108,110,116,119,125,126,129,132,133,138,140,142,143,151,152]

In [20]:
for i in add_ints:
    # Set a cell value in status
    unk_df.at[i, 'status'] = 'k'

In [23]:
translations = []
for i in range(0,len(unk_df)):
    try:
        translations.append(ts.google(unk_df.word.iloc[i]))
    except:
        translations.append('')
unk_df['translations']=translations

In [24]:
mitosheet.sheet(unk_df, analysis_to_replay="id-ianxevwttg")

MitoWidget(analysis_data_json='{"analysisName": "id-ianxevwttg", "analysisToReplay": null, "code": [], "stepSu…

In [21]:
add_words = list(unk_df.word[unk_df['status']=='k'])
print(len(add_words))
for word in add_words:
    vocab['white_listed'].add(word)

51


In [53]:
filtered_art = filter_text(textfile,text= True)

In [57]:
filtered_art['theme']=['' for line in filtered_art.line]
filtered_art = filtered_art[['theme','unk_words','line','line_percent']]
with open(path+'dad_choose_themes.xlsx','wb') as outfile:
    filtered_art.to_excel(outfile, encoding='utf-8',index=False)

In [63]:
with open(path+'dad_choose_themes.xlsx','rb') as infile:
    filtered_art = pd.read_excel(infile)

In [66]:
themes = set(filtered_art.theme[filtered_art['theme']!='NaN'])
output = pd.DataFrame(columns=['theme','line'])
for theme in themes:
    temp = filtered_art[['theme','line']][filtered_art['theme']==theme]
    output = pd.concat([output,temp])
with open(path+'dad_article.xlsx','wb') as outfile:
    output.to_excel(outfile, encoding='utf-8',index=False)

,theme,line
13,football,Et puis nous parlerons de sport de la Coupe du monde de football.
14,football,L'Allemagne se fait surprendre par le Japon.
15,football,Les Allemands battus par les Japonais.
17,football,Un match de foot marqué par une image.
18,football,Une image très forte.
...,...,...
21,ukraine,"La guerre en Ukraine et de nouveaux bombardements russes, bombardements qui ont fait au moins trois morts et six blessés à Kiev."
25,ukraine,"Correspondant régional de Radio France Internationale, Stéphane, trois centrales nucléaires ukrainiennes ont été déconnectées, conséquence?"
26,ukraine,Plus d'électricité dans une grande partie du pays.
37,ukraine,"Et puis à noter Adrien que le président français va une nouvelle fois parler au président russe\n\nEmmanuel Macron annonce qu'il aura un « contact direct » avec Vladimir Poutine, et ce, « dans les prochains jours » pour notamment parler de la centrale nucléaire de Zaporijia centrale nécessaire pour avoir accès à l'électricité en Ukraine."


In [68]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(vocab['white_listed']))

13427


In [69]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13427


##Filter for lessons

In [10]:
with open(path+"text_input_dad_fr.txt","r") as infile:
    textfile = infile.read()
filtered_art = filter_text(textfile,text= True)
#display(filtered_art)

In [13]:
with open(path+'unknown_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [re.sub("^['|\"]|^\['|['|\"]$|'\]$","",line.strip().lower()) for line in new_words] #update regex
#new_words
print(len(new_words))
known_manual = get_known(new_words)

16
cou
Known =k
15 remaining
poitrine
Known =k
14 remaining
rassuré
Known =k
13 remaining
vaste
Known =k
12 remaining
tatouages
Known =k
11 remaining
souvenirs
Known =k
10 remaining
maladie
Known =k
9 remaining
seins
Known =k
8 remaining
aliments
Known =k
7 remaining
réuni
Known =k
6 remaining
genoux
Known =k
5 remaining
bizarre
Known =kk
got 1
4 remaining
colline
Known =k
3 remaining
–
Known =kk
got 2
2 remaining
indigènes
Known =k
1 remaining
lors
Known =k
0 remaining


In [14]:
print(len(known_manual))
for word in known_manual:
    vocab['white_listed'].add(word)

2


In [19]:
#known_lines_only = list(set(list(filtered_art.knowns[filtered_art.knowns != "..."])))
#known_lines_only = sorted(known_lines_only,key=len)
filtered_art = filter_text(textfile,text= True)
known_lines_only = list(filtered_art.line[filtered_art['line_percent']==1.0])
known_lines_only = sorted(known_lines_only,key=len)


In [20]:
#export knowns: [filtered_art.knowns != "..."]
#known_list = list(filtered_art.knowns)
with open(path+"known_lines_dad_fr.txt","w") as outfile:
    for line in known_lines_only:
        outfile.write(line+"\n")

In [21]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(vocab['white_listed']))

13376


In [361]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13199


##EB section

In [70]:
#read eb unknown files from .json formats
"""
Remaining vocab size = 2325
Total learned = 31
"""
with open(path+'eb_unk.json', "r") as infile:
    eb_unk = json.loads(infile.read())
print("Remaining vocab size = "+str(len(eb_unk)))

with open(path+'eb_unk_mo.json', "r") as infile:
    eb_unk_mo = json.loads(infile.read())

with open(path+'eb_learned.json', "r") as infile:
    eb_learned= json.loads(infile.read())
print("Total learned = "+str(len(eb_learned)))

with open(path+'eb_unk_examples.json', "r") as infile:
    eb_unk_examples= json.loads(infile.read())



Remaining vocab size = 2322
Total learned = 43


In [80]:
"""
space = plain, add without conjurgating or changing
c = conjugate as normal verb
x = noun that adds x when plural
s = noun that add s when plural
iv = inner verb. There is a verb in the expression that should be 
    conjurgated
"""
with open(path+"fr_eb_extra.txt","r") as infile:
    extras_lemmas = infile.read().split('\n')

for lemma in extras_lemmas:
    extras_morphs = how_to_add(lemma)
    
    eb_unk[lemma] = extras_morphs
    for morph in extras_morphs:        
        if morph not in eb_unk_mo:
            eb_unk_mo[morph] = lemma

print("Remaining vocab size = "+str(len(eb_unk)))

riposte
c
x
s
ivs
davantage 
c
x
s
iv
peiné
c
x
s
ivs
briguer
c
x
s
ivc
rien n’est joué 
c
x
s
iv
ne date pas d’hier
c
x
s
iv
agacer 
c
x
s
ivc
autochtones
c
x
s
iv
d'aplomb
c
x
s
iv
âpre
c
x
s
ivs
Remaining vocab size = 2332


Get larger list of pages to search

In [81]:
websites = set()
with open(path+'viewed_websites_fr.txt',"r") as infile:
    for line in infile.read().split('\n'):
        websites.add(line)
print(len(websites))

152


In [71]:
pages = []

In [72]:
##France24
print("Checking France24")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.france24.com/fr/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))



for link in soup.find_all('a', href=True):
    str_ver = str(link['href'])
    
    #get regex such that links start with /fr/ and don't end
    #with /
    if bool(re.search('^\/fr\/.*',str_ver)) and (str_ver[len(str_ver)-1]!='/'):
        pages.append('https://www.france24.com'+str_ver)

Checking France24


In [73]:
##20minutes
print("Checking 20minutes")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.20minutes.fr/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    str_ver = str(link['href'])
    
    final_segment = str_ver.split('/')[len(str_ver.split('/'))-1]
    #articles have https:// at the beginning and a date in the last segment
    if ('https://www.20minutes.fr' in str_ver)\
    and bool(re.search('[0-9]',final_segment)):
        pages.append(str_ver)

Checking 20minutes


In [74]:
##TV5Monde
print("Checking TV5Monde")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.tv5monde.com/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    if '/info/' in str(link['href']):
        #print(str(link['href']))
        #str_page = 'https://savoirs.rfi.fr/'+str(link['href'])
        #str_page = 'https://savoirs.rfi.fr/en/apprendre-enseigner/langue-francaise/journal-en-francais-facile-'+str(re.search(r"[0-9]{8}.*",str(link['href'])).group(0))
        if str(link['href']) not in pages:
            pages.append(str(link['href']))


Checking TV5Monde


In [75]:
pages = list(set(pages))
len(pages)
#add in set_difference line here

151

In [79]:
with open(path+'viewed_websites_fr.txt',"w") as outfile:
    for line in websites:
        outfile.write(line+'\n')      

In [86]:
out = find_unks(pages,eb_unk_mo,eb_unk_examples)

In [ ]:
review_out(out,eb_learned,eb_unk,eb_unk_mo,eb_unk_examples)

Lemma: val
Morph: ['val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', 'val', '

options : k,m,a,q
Imprimer                                              Retourner en haut de la page        Annonces Légales   Annonces légales  Allier  Alpes Maritimes  Bas Rhin  Bouches du Rhône  Côtes d'Armor  Finistère  Haut Rhin  Hauts de Seine  Loire  Loire Atlantique  Loiret  Maine et Loire  Marne  Paris  Seine Maritime  Val d'Oise  Val de Marne  Var  Vaucluse  Yvelines      Services   Distribution  Horoscope  Horoscope chinois  Conjugaison  Recette de cuisine  Résultats des élections  Maison  Le Journal des Seniors  Bébés et Mamans  Infolocale  Sportune   Codes promo   Codes promo  Code promo AliExpress  Code promo Carrefour  Code promo Dell  Code promo Vistaprint  Code promo Black Friday     20 Minutes   Recrutement  La 21 e Minute  20 Minutes Média  Panel des lecteurs  20 Minutes Livres  Charte   Jeux   Jeux gratuits  Mots fléchés  Solitaire  Puzzle  Mots croisés  Sudoku      Réseaux sociaux        Facebook  2,9M         Twitter  2.4M         Instagram  320K         Youtube  

In [85]:
#write to eb unknown files to .json formats
"""
Remaining vocab size = 2322
Total learned = 43
"""
with open(path+'eb_unk.json', "w") as outfile:
    json.dump(eb_unk,outfile)
print("Remaining vocab size = "+str(len(eb_unk)))

with open(path+'eb_unk_mo.json', "w") as outfile:
    json.dump(eb_unk_mo,outfile)

with open(path+'eb_learned.json', "w") as outfile:
    json.dump(eb_learned,outfile)
print("Total learned = "+str(len(eb_learned)))

with open(path+'eb_unk_examples.json', "w") as outfile:
    json.dump(eb_unk_examples,outfile)

Remaining vocab size = 2322
Total learned = 43


### Code no longer in use

In [177]:
def return_line_percents(webpage):
    request=urllib.request.Request(webpage,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read()
    contents = text_from_html(data)

    known_array = []
    unk_array = []
    contents_array = sent_tokenize(contents)

    lines = []

    for line in contents_array:
        tokenized = word_tokenize(line, language='french')

        line_total = 0
        line_unks = 0
        for word in tokenized:

            line_total +=1

            if not (bool(re.search("[0-9]", word)) or\
                    bool(re.search("[A-Z]", word)) or\
                    bool(re.search("[.,\/#!$%\^&\*;:{}=\-_`~()«»]", word)) or\
                    word.lower() in vocab['white_listed']):

                    line_unks +=1

            lines.append(100*(line_total-line_unks)/line_total)
    return(lines)

In [ ]:

if (start is not None) and (stop is not None):
    contents_array=contents_array[max(start,0):min(stop,len(contents_array))]

with open(path+'unknown_french_dad_list.txt',"w") as outfile:
    outfile.write(str(list(disallowed_words)))


In [300]:
##add in extras
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/fr_eb_extra.txt","r") as infile:
    extras_lemmas = infile.read().split('\n')

gs = goslate.Goslate()

default_conjugator = mlconjug3.Conjugator(language='fr')


for lemma in extras_lemmas:
    
    decision = how_to_add(lemma)
    
    if lemma not in eb_unk:
        eb_unk[lemma] = []
        
    extras_morphs = []
    extras_morphs.append(lemma)
    try:
        #if it's an infinitive, add all conjurgations
        test_verb = default_conjugator.conjugate(lemma)
        all_conjugated_forms = test_verb.iterate()
        for item in all_conjugated_forms:
            if item not in extras_morphs:
                extras_morphs.append(item[len(item)-1])
        extras_morphs = list(set(extras_morphs))
    except:
        
        #if not an expression:
        if len(lemma.split(' '))==1:
            #add plural nouns
            if lemma[len(lemma)-1]=='u':
                extras_morphs.append(lemma+'x')
            else:
                extras_morphs.append(lemma+'s')

    eb_unk[lemma] = extras_morphs
    for morph in extras_morphs:        
        if morph not in eb_unk_mo:
            eb_unk_mo[morph] = lemma

print("Remaining vocab size = "+str(len(eb_unk)))

Remaining vocab size = 2327


In [8]:
#Upload from download
with open(path+'known_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [line.replace('"',"").strip().lower() for line in new_words]
    print("daily catch in word count:")
    print(len(new_words))
    
    for line in new_words:   
        vocab['white_listed'].add(line.replace('"',"").strip().lower())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/known_french_dad_list.txt'

In [117]:
"""
to do:

"""

In [49]:
##INPUT-OUTPUT
#vocab list:
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/dad_whitelisted.csv","r") as infile:
    whitelisted_lemmas = infile.read()


##VARIABLES
vocab_all = set()
gs = goslate.Goslate()

default_conjugator = mlconjug3.Conjugator(language='fr')


for line in whitelisted_lemmas.split('\n'):
    if len(line) > 0:
        #print(line)
        vocab = line.lower().strip()
        vocab_all.add(vocab)
        try:
            #if it's an infinitive, add all conjurgations
            test_verb = default_conjugator.conjugate(vocab)
            all_conjugated_forms = test_verb.iterate()
            for item in all_conjugated_forms:
                
                vocab_all.add(item[len(item)-1])
                
            
        except:
            #might be a noun, add plural
            vocab_all.add(add_noun(vocab))
#len(whitelisted_lemmas.split('\n'))            
#pp.pprint(vocab_all)

In [83]:
vocab = {}
vocab['white_listed'] = list(vocab_all)
vocab['black_listed'] = []


In [27]:
#used for creating eb data

with open(path+'eb_unk_lemmas.csv','r') as infile:
    eb_unk_lemmas = pd.read_csv(infile)
with open(path+'eb_unk_morph.csv','r') as infile:
    eb_unk_morph = pd.read_csv(infile)
    
eb_unk = {}
for word in eb_unk_lemmas.lemme:
    eb_unk[word] = []

    for index, row in eb_unk_morph.iterrows():
        if row.lemme == word:
            eb_unk[word].append(row.ortho)
eb_unk_mo = {}
for lemma in eb_unk.keys():
    for morph in eb_unk[lemma]:
        eb_unk_mo[morph] = lemma

In [147]:
eb_learned = {}
eb_unk_examples = {}
for word in eb_unk.keys():
    eb_unk_examples[word] = []

In [130]:
#restore to backups
eb_learned = eb_learned_backup
eb_unk = eb_unk_backup
eb_unk_mo = eb_unk_mo_backup
eb_unk_examples = eb_unk_examples